In [1]:
import numpy as np
import math
from collections import defaultdict

In [2]:
def load_sequences(filename):
    with open(filename, 'r') as f:
        raw_data = [list(map(int, line.strip().split())) for line in f]
    sequences = [row[1:] for row in raw_data]  # ignore user_id
    return sequences

In [3]:
def split_sequences_leave_two_out(sequences):
    train_seqs, val_items, test_items = [], [], []
    for seq in sequences:
        if len(seq) < 3:
            continue  # Need at least 3 items cv
        train_seqs.append(seq[:-2])      # all except last two
        val_items.append(seq[-2])        # second last
        test_items.append(seq[-1])       # last
    return train_seqs, val_items, test_items

In [4]:
def build_transition_matrix(sequences):
    matrix = defaultdict(lambda: defaultdict(int))
    for seq in sequences:
        for i in range(len(seq) - 1):
            curr_item = seq[i]
            next_item = seq[i + 1]
            matrix[curr_item][next_item] += 1
    return matrix

In [5]:
def predict_markov_chain(input_seq, transition_matrix):
    last_item = input_seq[-1]
    if last_item not in transition_matrix:
        return []
    next_items = transition_matrix[last_item]
    sorted_items = sorted(next_items.items(), key=lambda x: x[1], reverse=True)
    return [item for item, _ in sorted_items]

In [6]:
def hit_at_k(predictions, true_item, k):
    return 1 if true_item in predictions[:k] else 0

def ndcg_at_k(predictions, true_item, k):
    if true_item in predictions[:k]:
        idx = predictions[:k].index(true_item)
        return 1 / math.log2(idx + 2)  # +2 because index starts at 0
    return 0

In [7]:
def evaluate_markov_chain_leave_two_out(train_seqs, val_items, test_items, transition_matrix):
    hit10, hit20, ndcg10, ndcg20 = [], [], [], []

    for train_seq, test_item in zip(train_seqs, test_items):
        if len(train_seq) < 1:
            continue
        predictions = predict_markov_chain(train_seq, transition_matrix)

        hit10.append(hit_at_k(predictions, test_item, k=10))
        hit20.append(hit_at_k(predictions, test_item, k=20))
        ndcg10.append(ndcg_at_k(predictions, test_item, k=10))
        ndcg20.append(ndcg_at_k(predictions, test_item, k=20))

    return {
        "Hit@10": np.mean(hit10),
        "Hit@20": np.mean(hit20),
        "NDCG@10": np.mean(ndcg10),
        "NDCG@20": np.mean(ndcg20),
    }

In [10]:
if __name__ == "__main__":
    sequences = load_sequences("Beauty_item_org_rank.txt")
    train_sequences, val_items, test_items = split_sequences_leave_two_out(sequences)
    transition_matrix = build_transition_matrix(train_sequences)

    print("Evaluating Beauty_item_org_rank.txt")
    results = evaluate_markov_chain_leave_two_out(train_sequences, val_items, test_items, transition_matrix)

    for metric, score in results.items():
        print(f"{metric}: {score:.4f}")

Evaluating Beauty_item_org_rank.txt
Hit@10: 0.0312
Hit@20: 0.0409
NDCG@10: 0.0176
NDCG@20: 0.0201


In [12]:
if __name__ == "__main__":
    sequences = load_sequences("Beauty_item_var_rank.txt")
    train_sequences, val_items, test_items = split_sequences_leave_two_out(sequences)
    transition_matrix = build_transition_matrix(train_sequences)

    print("Evaluating Beauty_item_var_rank.txt")
    results = evaluate_markov_chain_leave_two_out(train_sequences, val_items, test_items, transition_matrix)

    for metric, score in results.items():
        print(f"{metric}: {score:.4f}")

Evaluating Beauty_item_var_rank.txt
Hit@10: 0.0322
Hit@20: 0.0428
NDCG@10: 0.0183
NDCG@20: 0.0210
